# Исследование надежности заемщиков.

Оценка кредитоспособности клиента или *кредитный скоринг* широко используется банками для того, чтобы быстро принять решение об одобрении или отказе в выдаче кредита. На основе данных мы сравним надежность людей с разным бэкраундом и выявим закономерности.

## Шаг 1. Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('data.csv')
display(df.head(10)) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Далее нам нужно изучить наш набор данных. Являются ли они завершенными? Распознало ли pandas их, как ожидалось? Отсутствуют ли в них какие-либо значения?

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице двенадцать столбцов. Тип данных в столбцах `education`, `family_status`, `gender`, `income_type` and `purpose` — `object`. В столбцах `days_employed` and `total_income` - `float64`. В остальных - `int64`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о клиенте банка. Часть колонок описывает взятый кредит: цель кредита и наличие или отсутствие по нему задолженносоти. Остальные данные рассказывают о самом клиенте: его семейное положение, пол, тип занятости, наличие детей и их количество, уровень образования и так далее.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но в данных встречаются пропуски.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Шаг 2.1 Заполнение пропусков

In [4]:
df.isna().sum()#узнаем сколько пропусков в каждом столбце датафрейма

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
days_employed_na = 2174
total_entries = 21525
f'Доля пропусков в таблице составляет {days_employed_na/total_entries:.1%}'

'Доля пропусков в таблице составляет 10.1%'

Мы видим, что в колонках `days_employed` и `total_income` одинаковое количество пропусков. Это может означать, что у человека без трудового стажа (days_employed=0) нет никакого дохода.

Доля пропусков составляет 10%, это небольшая погрешность. Пропущенные значения неважны для целей исследования, но в тех же строках или столбцах остаются ценные данные, поэтому простое удаление строк нам не подходит. Заполнить пропуски медианным значением — лучшее решение для наших данных.

Для начала узнаем, какой тип занятости у клиентов без дохода.

In [6]:
#фильтруем по нулевым значениям и выбираем список из уникальных профессий
display(df[df['total_income'].isna()]['income_type'].unique())

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Мы увидели следущие профессии:
* пенсионер
* госслужащий
* компаньон
* сотрудник
* предприниматель

Для каждого типа найдем медианное значение и заполним им ячейки с нулями.

Создадим функцию `profession_median()` с параметром `profession` - профессиями, у которых встречаются пропуски в доходах.

Функция должна вернуть медианное значение по доходам этих профессий.

In [7]:
def profession_median(profession):
    profession_list = df[(df['income_type'] == profession) & (df['total_income'] != 0)]  #выбираем профессии  с пропусками в данных
    median_count = profession_list['total_income'].median() #находим медианное значение
    return median_count

Подставим в функцию наши профессии и проверим, что функция работает

In [8]:
profession_median('пенсионер')

118514.48641164352

In [9]:
profession_median('госслужащий')

150447.9352830068

In [10]:
profession_median('компаньон')

172357.95096577113

In [11]:
profession_median('сотрудник')

142594.39684740017

In [12]:
profession_median('предприниматель')

499163.1449470857

Доход предпринимателя в 4 раза выше дохода пенсионера - похоже на правду.

Теперь заменим все пропуски на полученные медианные значения:

In [13]:
professions = df[df['total_income'].isna()]['income_type'].unique() #список с профессиями, у которых найдены пропуски

for profession in professions:
    df.loc[df['income_type'] == profession, 'total_income'] = df[df['income_type'] == profession]['total_income'].fillna(profession_median(profession))

In [14]:
#проверим результат
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

**Вывод:** чтобы заполнить пропуски в `total_income` мы нашли мединные значения разных групп из `income_type`. Чтобы исправить пропуски в `days_employed` сперва нужно найти и исправить аномалии.

### Шаг 2.2 Проверка данных на аномалии и исправления.

В столбце `days_employed` присутвуют отрицательные значения. Предположим, что при нахождении числа человек перепутал столбцы и вычел из даты приёма дату увольнения. Тогда функция `df.abs()` исправит ситуацию.

In [15]:
for index in df:
    try:
        df['days_employed'] = df['days_employed'].abs()
    except:
        print(row[index])

<div class="alert alert-block alert-success">
<b>Комментарий ревьюера (✅):</b> Пресловутый человеческий фактор :)

Также мы видим что в данных есть значения >20000 дней, это больше 55 лет. Посчитаем количество строк с такими даннымх.

In [16]:
df.sort_values(by='days_employed', ascending=False)
df[df['days_employed'] > 20000]['days_employed'].count()

3445

Чтобы заменить эти данные, сделаем функцию, которая возвращает возрастную группу по значению возраста age, используя правила:

    - 'дети', если age <= 18 лет;
    - 'взрослые', если age от 19 до 64;
    - 'пенсионеры' — от 65 и старше.

In [17]:
def age_group(age):
   
    if age <= 18:
        return 'дети'
    if age <= 64:
        return 'взрослые'
    return 'пенсионеры'

df['age_group'] = df['dob_years'].apply(age_group) #добавляем данные в столбец age_group 
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,взрослые
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,пенсионеры
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,взрослые
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,взрослые
21524,2,1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля,взрослые


Найдём медианное значение `days_employed`для каждой категории

In [18]:
stop_working_after_death = df[df['days_employed'] < 20000]
age_group = stop_working_after_death.groupby('age_group')['days_employed'].median()
age_group

age_group
взрослые      1626.335854
дети          1560.900431
пенсионеры    2876.221697
Name: days_employed, dtype: float64

Согласно предоставленным данным у детей средний стаж не уступает стажу взрослых. Это невозможно, проверим, нет ли пропусков в данных:

In [19]:
print(df[df['age_group'] == 'дети']['dob_years'].unique())

[0]


Действительно в категорию `дети` попали люди с нулевым возрастом. Заполним нули средним возрастом взрослых. Ранее мы убедились в том, что у этих двух категорий похожий медианный `days_employed`. Затем переименуем категорию `дети` в категорию `взрослые`.

In [20]:
ad_median = df[df['age_group'] == 'взрослые']['dob_years'].median() #найдем медианный взраст всех взрослых
df.loc[df['dob_years'] == 0,'dob_years'] = ad_median 

In [21]:
df[df['dob_years'] == 0]['dob_years'].count() #проверим, что пропусков в возрасте не осталось.

0

In [22]:
df.loc[df['age_group'] == 'дети','age_group'] = 'взрослые' #переменуем всех детей во взрослых.

Осталось заменить значения `days_employed`, которые превышают 55 лет. Так же заменим их на медианные значения по категориям `взрослые` и  `пенсионеры`.

In [23]:
stop_working_after_death = df[df['days_employed'] < 20000]
age_group = stop_working_after_death.groupby('age_group')['days_employed'].median().reset_index()
age_group

,age_group,days_employed
0,взрослые,1624.965472
1,пенсионеры,2876.221697


In [24]:
df.loc[df['age_group'] == 'взрослые', 'days_employed'] = df[df['age_group'] == 'взрослые']['days_employed'].fillna(age_group.loc[0,'days_employed'])
df.loc[df['age_group'] == 'пенсионеры','days_employed'] = df[df['age_group'] == 'пенсионеры']['days_employed'].fillna(age_group.loc[1,'days_employed'])
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

In [25]:
df.loc[(df['days_employed'] > 20000) & (df['age_group'] == 'взрослые'),'days_employed'] = age_group.loc[0,'days_employed']
df.loc[(df['days_employed'] > 20000) & (df['age_group'] == 'пенсионеры'), 'days_employed'] = age_group.loc[1,'days_employed']
df[df['days_employed'] > 20000]['days_employed'].count()

0

In [26]:
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,взрослые
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,взрослые
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,взрослые
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,взрослые
4,0,1624.965472,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,взрослые


Теперь проверим аномалии в других столбцах. Например, сколько уникальных вариантов содержит столбец `gender`?

In [27]:
df.gender.unique()

array(['F', 'M', 'XNA'], dtype=object)

In [28]:
#найдем индексы гендера XNA
df[df['gender'] == 'XNA'][index] 

10701    покупка недвижимости
Name: purpose, dtype: object

In [29]:
#инедекс всего один, удалим строчку без данных.
df.drop(index=[10701], axis = 0, inplace =True)
df.gender.unique() #проверим, что гендера всео два

array(['F', 'M'], dtype=object)

Теперь проверим в каком диапазоне представлено количество детей.

In [30]:
df.groupby('children')['days_employed'].count()

children
-1        47
 0     14148
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: days_employed, dtype: int64

Отрицательное количество детей - маловероятно, скорее всего человек поставил тире перед числом. Заменим его на модуль числа. 

Другая аномалия - у 76 человек по 20 детей. Я бы уточнила у коллеги, так ли это. Пока предположим, что у этого числа есть лишний ноль и добавим этих людей к группе людей, у которых всего по 2 ребенка.

In [31]:
for index in df:
    try:
        df['children'] = df['children'].abs()
    except:
        print(row[index])

In [32]:
df.groupby('children')['days_employed'].count()

children
0     14148
1      4865
2      2055
3       330
4        41
5         9
20       76
Name: days_employed, dtype: int64

In [33]:
df.loc[df['children'] == 20,'children'] = 2

In [34]:
df.groupby('children')['days_employed'].count()

children
0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: days_employed, dtype: int64

**Вывод:** Мы заполнили недостающие данные и проработали аномальные значения в столбцах `dob_years`, `days_employed`,`total_income`, `gender` и `children`

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбцах `total_income` и `days_employed` на целочисленный

In [35]:
df.astype({'total_income': 'int32', 'days_employed': 'int32'}).dtypes

children              int64
days_employed         int32
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int32
purpose              object
age_group            object
dtype: object

### Шаг 2.4. Удаление дубликатов.

Теперь избавимся от неявных дубликатов в колонке `education`. Например, название одной и той же степени может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

In [36]:
df['education'].sort_values().unique() # Просмотр уникальных видов образования

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Мы видим следующие неявные дубликаты:

* 'ВЫСШЕЕ', 'Высшее' и 'высшее'
* 'НАЧАЛЬНОЕ', 'Начальное' и 'начальное'
* 'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее' и 'неоконченное высшее'
* 'СРЕДНЕЕ', 'Среднее' и 'среднее'
* 'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень' и 'ученая степень

Приведем всё к одному регистру.

In [37]:
df['education'] = df['education'].str.lower()
df['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два словаря - образования и семейного  положения - к которым мы сможем обращаться по идентификатору. 

Каждому уникальному значению из education соответствует уникальное значение education_id

In [38]:
education_df = df[['education','education_id']]
education_df = education_df.drop_duplicates().reset_index(drop=True)
education_df.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Каждому уникальному значению из family_status соответствует уникальное значение family_status_id

In [39]:
family_satus_df = df[['family_status','family_status_id']]
family_satus_df = family_satus_df.drop_duplicates().reset_index(drop=True)
family_satus_df.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [40]:
df.drop(['education','family_status'], axis = 1, inplace=True) #удалим столбцы `education` и `family_status`
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42.0,0,0,F,сотрудник,0,253875.639453,покупка жилья,взрослые
1,1,4024.803754,36.0,1,0,F,сотрудник,0,112080.014102,приобретение автомобиля,взрослые
2,0,5623.422610,33.0,1,0,M,сотрудник,0,145885.952297,покупка жилья,взрослые
3,3,4124.747207,32.0,1,0,M,сотрудник,0,267628.550329,дополнительное образование,взрослые
4,0,1624.965472,53.0,1,1,F,пенсионер,0,158616.077870,сыграть свадьбу,взрослые


### Шаг 2.6. Категоризация дохода.

Составим новый столбец с категориями дохода. Сделаем функцию, которая возвращает категорию по значению `total_income`, используя правила:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'

In [41]:
def total_income_category(income):
   
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

df['total_income_category'] = df['total_income'].apply(total_income_category) #добавляем данные в столбец total_income_category 
df.tail()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,age_group,total_income_category
21520,1,4529.316663,43.0,1,1,F,компаньон,0,224791.862382,операции с жильем,взрослые,B
21521,0,2876.221697,67.0,1,0,F,пенсионер,0,155999.806512,сделка с автомобилем,пенсионеры,C
21522,1,2113.346888,38.0,1,1,M,сотрудник,1,89672.561153,недвижимость,взрослые,C
21523,3,3112.481705,38.0,1,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,взрослые,B
21524,2,1984.507589,40.0,1,0,F,сотрудник,0,82047.418899,на покупку автомобиля,взрослые,C


### Шаг 2.7. Категоризация целей кредита.

Объеденим похожие цели кредита в одну категорию. Для этого создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

In [42]:
df['purpose'].sort_values().unique() #перечислим уникальыне цели кредита

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [43]:
def purpose_category(purpose):
   
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'образовани' in purpose:
        return 'получение образования'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    return 'проведение свадьбы'

df['purpose_category'] = df['purpose'].apply(purpose_category) #добавляем данные в столбец purpose_category 

### Шаг 3. Ответы на вопросы

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Составим сводную таблицу, в которой найдем отношение кредитов погашенных (сумм по столбцу debt) к общему числу взятых  кредитов (количество по любому другому столбцу, например, dob_years).

In [44]:
data_pivot = df.pivot_table(index=['children'], values=['debt','dob_years'], aggfunc={'debt': 'sum','dob_years': 'count'}).reset_index()
data_pivot['%_of_total'] = data_pivot.debt / data_pivot.dob_years * 100
data_pivot.rename(columns={"debt": "заемщики_с_просроченным_платежом", "children": "детей_у_заемщика","dob_years": "всего_заемщиков", "%_of_total": "%_должников_среди_заемщиков"})

,детей_у_заемщика,заемщики_с_просроченным_платежом,всего_заемщиков,%_должников_среди_заемщиков
0,0,1063,14148,7.513429
1,1,445,4865,9.146968
2,2,202,2131,9.479118
3,3,27,330,8.181818
4,4,4,41,9.756098
5,5,0,9,0.000000


In [45]:
data_pivot.query('children > 0 and children < 5')['%_of_total'].mean() #средняя доля должников среди заемщиков, у которых от 1 до 4 детей.

9.141000416911279

##### Вывод 1:

Клиенты без детей берут кредиты гораздо чаще, чем клиенты с детьми. Среди таких заемщиков процент по невозвратам самый низкий - 7.5%. У заемщиков с количеством детей до четырех средний процент просрока 9.14%, хуже всего обстоят дела у заёмщикв с 4 детьми - 9.75%. Клиенты с 5 детьми - самая малочисленная группа, однако, ни один клиент, у которого 5 детей, не просрочил кредит.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [46]:
data_pivot = df.pivot_table(index=['family_status_id'], values=['debt','dob_years'], aggfunc={'debt': 'sum','dob_years': 'count'}).reset_index()
data_pivot['%_of_total'] = data_pivot.debt / data_pivot.dob_years * 100
data_pivot.rename(columns={"family_status_id": "семейный статус", "dob_years": "всего_заемщиков", "%_of_total": "%_должников_среди_заемщиков"})

,семейный статус,debt,всего_заемщиков,%_должников_среди_заемщиков
0,0,931,12380,7.520194
1,1,388,4176,9.291188
2,2,63,960,6.562500
3,3,85,1195,7.112971
4,4,274,2813,9.740491


In [47]:
data_pivot = df.pivot_table(index=['family_status_id'], values=['debt','dob_years'], aggfunc={'debt': 'sum','dob_years': 'count'}).reset_index()
data_pivot['%_of_total'] = data_pivot.debt / data_pivot.dob_years * 100
data_pivot.rename(columns={"family_status_id": "семейное положение","debt": "заемщики_с_просроченным_платежом", "dob_years": "всего_заемщиков", "%_of_total": "%_должников_среди_заемщиков"})

,семейное положение,заемщики_с_просроченным_платежом,всего_заемщиков,%_должников_среди_заемщиков
0,0,931,12380,7.520194
1,1,388,4176,9.291188
2,2,63,960,6.562500
3,3,85,1195,7.112971
4,4,274,2813,9.740491


##### Вывод 2:

- 0 женат / замужем
- 1 гражданский брак
- 2 вдовец / вдова
- 3 в разводе
- 4 Не женат / не замужем

Неженатые клиенты - самые ненадежные заёмщики, 9.7% не возвращают вредит вовремя. Пары с общим бытом также имеет высокий процент невозврата, 9.3%. Самые ответсвенные заёмщики - вдовы или вдоцы, всего 6.6% имет задолженность.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
data_pivot = df.pivot_table(index=['total_income_category'], values=['debt','dob_years'], aggfunc={'debt': 'sum','dob_years': 'count'}).reset_index()
data_pivot['%_of_total'] = data_pivot.debt / data_pivot.dob_years * 100
data_pivot.rename(columns={"total_income_category": "категория_дохода","debt": "заемщики_с_просроченным_платежом", "dob_years": "всего_заемщиков", "%_of_total": "%_должников_среди_заемщиков"})

,категория_дохода,заемщики_с_просроченным_платежом,всего_заемщиков,%_должников_среди_заемщиков
0,A,2,25,8.000000
1,B,356,5041,7.062091
2,C,1360,16086,8.454557
3,D,21,350,6.000000
4,E,2,22,9.090909


##### Вывод 3:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'

Клиенты с доходом до 30000 берут кредит реже всех, и чаще других не возвращают в срок. Клиенты с доходом от 50001 до 200000 берут кредиты часто, и около 8,5% не возвращает вовремя. Данные говорят, что заемщики категории D самые ответственные. Прямой зависимости между уровнем дохода и возвратом кредита нет.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [49]:
data_pivot = df.pivot_table(index=['purpose_category'], values=['debt','dob_years'], aggfunc={'debt': 'sum','dob_years': 'count'}).reset_index()
data_pivot['%_of_total'] = data_pivot.debt / data_pivot.dob_years * 100
data_pivot.rename(columns={"purpose_category": "цель_кредита","debt": "заемщики_с_просроченным_платежом", "dob_years": "всего_заемщиков", "%_of_total": "%_должников_среди_заемщиков"})

,цель_кредита,заемщики_с_просроченным_платежом,всего_заемщиков,%_должников_среди_заемщиков
0,операции с автомобилем,403,4315,9.339513
1,операции с недвижимостью,782,10839,7.214688
2,получение образования,370,4022,9.199403
3,проведение свадьбы,186,2348,7.921635


##### Вывод 4:

Кредиты на на свадьбу или операции с недвижимостью имеют больший процент возврата, чем кредиты операции с автомобилем и на получение образования. 

## Общий вывод:

Целю нашей работы является портрет идеального потенциального заёмщика. 

Для того, чтобы его найти, нужно было обработать данные о должниках и добросовестных кредиторах, полученные от кредитного отдела банка.

Данные по некоторым показателям нуждались в предбработке. В таблице нашлись аномалии, такие как пропуски значений, явные и нявные дубликаты, опечатки, дробные числа. Всё это нужно было исправить и подготовить для дальнейшей работы.

Чтобы не обращаться к огромной таблице, мы прибегнули к категоризации и создали несколько групп, объединенных по общему признаку. так мы объединили людей с похожим уровнем дохода, и разделили все цели кредита на четыре основных.

Далее мы провели 4 гипотезы и установили:

1. В начале мы предположили, что женатые люди с детьми погашают кредит в срок. Большее количество детей также увеличивает шанс возврата кредита. Однако, **наличие детей влияет на возвратность кредита отрицательно**, клиенты без детей не только берут кредиты гораздо чаще, чем клиенты с детьми, но и просрачивают реже. Однако, среди клиентов с 5 детьми (самая малочисленная группа) не было ни одного просрока. Но, их выборка мала, опираться на аткие данные непарвильно. Эта гипотеза подтвердилась частично.
2. **Наличие официального партнера благотворительно влияет на возвратность кредита**, однако, самые ответсвенные заёмщики - вдовы или вдовцы. Это частично подтвердило нашу гипотезу
3. Данные говорят, что заемщики со средним доходом 30001–50000 самые ответственные. Даже у людей с очень высоким доходом процент просрока хуже. Наша гипотеза не подтвердилась,  **прямой зависимости между уровнем дохода и возвратом кредита нет**.
4. **Кредиты на на свадьбу или операции с недвижимостью имеют больший процент возврата**, чем кредиты операции с автомобилем и на получение образования, что подтверждает гипотезу о том, что цель кредита тоже имеет значение в портете идеального заемщика.

Кредитному отделу банка при построении модели кредитного скоринга нужно опираться на следующих людей:
- заёмщиков без детей
- вдоцов,вдов, людей в браке или отношениях
- доход обязателен, но размер не влияет
- заемщиков, которые берут кредит на свадьбу или операции с недвижимостью.